In [ ]:
#LCG
# Inverse Transform


In [26]:
import numpy as np
import math

In [27]:
def LCG(N):
    uniformSamples = []
    x = 1
    a = 39373
    c = 0
    k = (2**31) - 1
    
    for i in range(N):
        x = ((a*x) + c)%k
        uniformSamples.append(x/k)
    
    return uniformSamples

In [28]:
def beaselySpringerMoro(u):
    # Constants
    a = [2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637]
    b = [-8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833]
    c = [0.3374754822726147, 0.9761690190917186, 0.1607979714918209,
         0.0276438810333863, 0.0038405729373609, 0.0003951896511919,
         0.0000321767881768, 0.0000002888167364, 0.0000003960315187]

    # Compute
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a[3] * r + a[2]) * r + a[1]) * r + a[0]) / ((((b[3] * r + b[2]) * r + b[1]) * r + b[0]) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c[0] + r * (c[1] + r * (c[2] + r * (c[3] + r * (c[4] + r * (c[5] + r * (c[6] + r * (c[7] + r * c[8])))))))
        if y < 0:
            x = -x

    return x

In [29]:
def inverseTransform(N):
    uniformSamples = LCG(N)
    normalSamples = []
    for i,u in enumerate(uniformSamples):
        normalSamples.append(beaselySpringerMoro(u))
    
    return normalSamples

In [57]:
numPaths = 1000000

In [58]:
T = 7/12
K = 52
S0 = 49
v = 0.27
r = 0.045
q = 0.02

In [59]:
def european_option_monte_carlo(S0, K, T, r, q, sigma, option_type, numPaths):
    dt = T
    Z = np.array(inverseTransform(numPaths))
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    
    if option_type == 'call':
        payoff = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoff = np.maximum(K - ST, 0)
    
    discount_factor = np.exp(-r * T)
    option_price = discount_factor * np.mean(payoff)
    return option_price

In [60]:
callP = european_option_monte_carlo(S0, K, T, r, q, v, 'call', numPaths)
print(callP)

3.048897492977414


In [61]:
putP = european_option_monte_carlo(S0, K, T, r, q, v, 'put', numPaths)
print(putP)

5.276422753272728


In [62]:
#Delta
deltaS = 1e-6
callUp = european_option_monte_carlo(S0 + deltaS, K, T, r, q, v, 'call', numPaths)
callDown = european_option_monte_carlo(S0 - deltaS, K, T, r, q, v, 'call', numPaths)

deltaCall = (callUp - callDown)/(2*deltaS)
print(deltaCall)

putUp = european_option_monte_carlo(S0 + deltaS, K, T, r, q, v, 'put', numPaths)
putDown = european_option_monte_carlo(S0 - deltaS, K, T, r, q, v, 'put', numPaths)

deltaPut = (putUp - putDown)/(2*deltaS)
print(deltaPut)

0.4491019749419678
-0.5391681154875982


In [63]:
#Gamma
#deltaS = 1e-6
#callUp = european_option_monte_carlo(S0 + deltaS, K, T, r, q, v, 'call', numPaths)
#callDown = european_option_monte_carlo(S0 - deltaS, K, T, r, q, v, 'call', numPaths)

#deltaCall = (callUp - callDown)/(2*deltaS)
#print(deltaCall)

#putUp = european_option_monte_carlo(S0 + deltaS, K, T, r, q, v, 'put', numPaths)
#putDown = european_option_monte_carlo(S0 - deltaS, K, T, r, q, v, 'put', numPaths)

#deltaPut = (putUp - putDown)/(2*deltaS)
#print(deltaPut)

gammaCall = (callUp + callDown - (2*callP))/(deltaS**2)
gammaPut = (putUp + putDown - (2*putP))/(deltaS**2)

print(gammaCall)
print(gammaPut)

0.0008881784197001252
0.005329070518200751


In [64]:
#Vega
deltaV = 1e-6
vegaUpC = european_option_monte_carlo(S0 , K, T, r, q, v + deltaV, 'call', numPaths)
vegaDownC = european_option_monte_carlo(S0 , K, T, r, q, v - deltaV , 'call', numPaths)

vegaCall = (vegaUpC - vegaDownC)/(2*deltaV)

vegaUpP = european_option_monte_carlo(S0 , K, T, r, q, v + deltaV, 'put', numPaths)
vegaDownP = european_option_monte_carlo(S0 , K, T, r, q, v - deltaV , 'put', numPaths)
vegaPut = (vegaUpP - vegaDownP)/(2*deltaV)

print(vegaCall)
print(vegaPut)

14.639904714286445
14.665115419809638


In [65]:
#Rho 
deltaR = 1e-6
rhoUpC = european_option_monte_carlo(S0 , K, T, r + deltaR, q, v , 'call', numPaths)
rhoDownC = european_option_monte_carlo(S0 , K, T, r - deltaR , q, v , 'call', numPaths)

rhoCall = (rhoUpC - rhoDownC)/(2*deltaR)

rhoUpP = european_option_monte_carlo(S0 , K, T, r + deltaR , q, v, 'put', numPaths)
rhoDownP = european_option_monte_carlo(S0 , K, T, r - deltaR , q, v, 'put', numPaths)
rhoPut = (rhoUpP - rhoDownP)/(2*deltaR)

print(rhoCall)
print(rhoPut)

11.058294217214382
-18.489149046896358


In [70]:
#theta
deltaT = 1e-9
thetaUpC = european_option_monte_carlo(S0 , K, T - deltaT, r , q, v , 'call', numPaths)
#thetaDownC = european_option_monte_carlo(S0 , K, T - deltaT, r  , q, v , 'call', numPaths)

thetaCall = (thetaUpC - callP)/(deltaT)

thetaUpP = european_option_monte_carlo(S0 , K, T - deltaT, r , q, v, 'put', numPaths)
#thetaDownP = european_option_monte_carlo(S0 , K, T - deltaT, r, q, v, 'put', numPaths)

thetaPut = (thetaUpP - putP)/(deltaT)

print(thetaCall/252)
print(thetaPut/252)

-0.01508349882385122
-0.009904785986101334


0.0027397260273972603